In [3]:
import attr
import enum

import pandas as pd
from hypothesis import *
from hypothesis import strategies as st

In [4]:
Synod = enum.Enum('Synod', 'NSWACT VICTAS QLD SA WA NT')

@attr.s
class Nominee:
    name = attr.ib(validator=attr.validators.instance_of(str))
    votes = attr.ib(validator=[attr.validators.instance_of(int), lambda _, __, ap: ap >= 0])
    synod = attr.ib(validator=attr.validators.instance_of(Synod))
    young = attr.ib(validator=attr.validators.instance_of(bool))
    ordained = attr.ib(validator=attr.validators.instance_of(bool))
    cald = attr.ib(validator=attr.validators.instance_of(bool))
    # For implementation reasons, the gender balance constraint is
    # represented as 10-14 men of 24 members
    male = attr.ib(validator=attr.validators.instance_of(bool))

In [5]:
Nominee.votes

Attribute(name='votes', default=NOTHING, validator=_AndValidator(_validators=(<instance_of validator for type <class 'int'>>, <function Nominee.<lambda> at 0x00000067C6F879D8>)), repr=True, cmp=True, hash=None, init=True, convert=None, metadata=mappingproxy({}))

In [6]:
nominee_strat = st.builds(Nominee, name=st.from_regex('\A[A-Z][a-z]{1,9} [A-Z][a-z]{1,9}\Z'),
                          votes=st.integers(0, 400), synod=st.sampled_from(Synod), young=st.booleans(),
                          ordained=st.booleans(), cald=st.booleans(), male=st.booleans())

In [7]:
nominee_strat.example()

Nominee(name='Fnehlyyrh Mynyimxf', votes=123, synod=<Synod.QLD: 3>, young=True, ordained=True, cald=True, male=True)

In [8]:
nominees = st.lists(nominee_strat, min_size=30, max_size=60, unique_by=lambda n: n.name)

In [10]:
import itertools

In [11]:
%%time
count = 0
it = itertools.combinations(sorted(nominees.example(), key=lambda n: n.votes)[:30], 18)
for _ in range(10**6):
    sum(n.votes for n in next(it))
    count +=1
print(count)

1000000
Wall time: 2.77 s


In [13]:
pool = sorted(nominees.example(), key=lambda n: n.votes, reverse=True)[:30]
[n.votes for n in pool]

[364,
 360,
 350,
 350,
 336,
 330,
 330,
 328,
 327,
 318,
 314,
 311,
 309,
 306,
 293,
 286,
 278,
 274,
 268,
 261,
 258,
 253,
 242,
 226,
 226,
 221,
 219,
 194,
 187,
 171]

In [ ]:
%%time
it = itertools.combinations(pool, 18)
best = next(it)
score = sum(n.votes for n in best)
for i, cte in enumerate(it):
    s = sum(n.votes for n in cte)
    if s > score:
        score = s
        best = cte
# time: ~2.8 seconds per million combinations
print(score, cte)